In [2]:
from pprint import pprint
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [5]:
import praw 
user_agent = "fekef"
reddit = praw.Reddit(
    client_id = "VafaxrhTsgDU_IC9z-8heA",
    client_secret = "IyYOKbjksZn4IcTSHk_JGKzYWZFtLw",
    user_agent = user_agent
)

In [17]:
headlines = set()
for submission in reddit.subreddit('climatechange').hot(limit=None):
    
    headlines.add(submission.title)
print(len(headlines))    

659


In [18]:
df = pd.DataFrame(headlines)
df.head()


,0
0,Hidden threat: Global underground infrastructu...
1,How exactly are we supposed to be fixing this?
2,England and Wales had warmest February on record
3,It’s so warm here in Chicago.
4,"Doomed polar bear population, perhaps."


In [19]:
df.to_csv("climate_headlines.csv", header = False, encoding = "utf-8", index = False)

In [20]:
import nltk

In [22]:
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/fekef/nltk_data...


True

In [23]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

In [27]:
sia = SIA()
results = []
for line in headlines: 
    pol_score = sia.polarity_scores(line)
    pol_score["headline"] = line 
    results.append(pol_score)
pprint(results[:3], width = 100)

[{'compound': -0.6486,
  'headline': 'Hidden threat: Global underground infrastructure vulnerable to sea-level rise',
  'neg': 0.431,
  'neu': 0.569,
  'pos': 0.0},
 {'compound': 0.0,
  'headline': 'How exactly are we supposed to be fixing this?',
  'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0},
 {'compound': 0.4019,
  'headline': 'England and Wales had warmest February on record',
  'neg': 0.0,
  'neu': 0.722,
  'pos': 0.278}]


In [28]:
df = pd.DataFrame.from_records(results)
df.head()

,neg,neu,pos,compound,headline
0,0.431,0.569,0.000,-0.6486,Hidden threat: Global underground infrastructu...
1,0.000,1.000,0.000,0.0000,How exactly are we supposed to be fixing this?
2,0.000,0.722,0.278,0.4019,England and Wales had warmest February on record
3,0.000,0.695,0.305,0.2944,It’s so warm here in Chicago.
4,0.512,0.488,0.000,-0.6369,"Doomed polar bear population, perhaps."


In [30]:
df["label"] = 0 
df.loc[df["compound"] > 0.2, "label"] = 1
df.loc[df["compound"] > -0.2, "label"] = -1
df.head()

,neg,neu,pos,compound,headline,label
0,0.431,0.569,0.000,-0.6486,Hidden threat: Global underground infrastructu...,0
1,0.000,1.000,0.000,0.0000,How exactly are we supposed to be fixing this?,-1
2,0.000,0.722,0.278,0.4019,England and Wales had warmest February on record,-1
3,0.000,0.695,0.305,0.2944,It’s so warm here in Chicago.,-1
4,0.512,0.488,0.000,-0.6369,"Doomed polar bear population, perhaps.",0


In [31]:
df2 = df[["headline", "label"]]